# Initialization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os, sys, time
import joblib

from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve, roc_curve, auc

from warnings import simplefilter
from category_encoders import TargetEncoder

# Ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" 

simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Specify database file location
db_filename = 'filepath'

# Determine file location for objects to be saved
cat_imputer_filename = 'filepath'
num_imputer_filename = 'filepath'
scaler_filename = 'filepath'
encoder_filename = 'filepath'

In [ ]:
# Read the excel file
df = pd.read_excel(db_filename)

In [ ]:
# Features are going to be the X variables and the Toxicity Labels the Y variable
X_pre = df.iloc[:,:-1]
y = df.loc[:, ['Target']]

In [ ]:
# Print Final DB Columns
X_pre.columns.tolist()

In [ ]:
# Encoding values
encoding_dict = {
    'Chemotherapy': {'N': 0, 'Y': 1},
    'Charlson': {'Low': 0, 'Medium': 1, 'High': 2, 'VeryHigh': 3},
    'KPS': {'Good': 0, 'Normal': 1, 'Bad': 2},
    'Ethnicity': {'C': 0, 'NC': 1},
    'TypeOfBoost': {'DidNotGet': 0, 'SIB': 1, 'Sequential': 2},
    'Applicator': {'Syed': 0, 'TO': 1},
    'MRI': {'N': 0, 'Y': 1},
    'TumorSite': {'Other': 0, 'Endometrium': 1, 'Cervix': 2},
    'Stage': {'I': 0, 'II': 1, 'III': 2, 'IV': 3},
    'Histology': {'SCC': 0, 'Non - SCC': 1},
}

categorical_columns = [
    'Chemotherapy', 'Charlson', 'KPS', 'Ethnicity',
    'TypeOfBoost', 'Applicator', 'MRI', 'TumorSite',
    'Stage', 'Histology'
]

# Determine the categorical column indexes to be used later
categorical_columns_index = list(map( lambda n: X_pre.columns.get_loc(n), categorical_columns) )

# Apply the encoding dictionary to the respective columns
for col, mapping in encoding_dict.items():
    X_pre[col] = X_pre[col].map(mapping)

# Train/Test split

In [ ]:
# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X_pre, y, random_state = 0, stratify = y, train_size = 0.75)

In [ ]:
print(f'Training set size -> {X_train.shape[0]}')
print(f'Testing set size -> {X_test.shape[0]}')

In [ ]:
print(f'Number of NEGATIVE cases in TRAIN -> {y_train.value_counts()[0]}')
print(f'Number of POSITIVE cases in TRAIN -> {y_train.value_counts()[1]}')
print(f'Ratio of P:N -> {y_train.value_counts()[1]/y_train.value_counts()[0]}')

In [ ]:
print(f'Number of NEGATIVE cases in TEST -> {y_test.value_counts()[0]}')
print(f'Number of POSITIVE cases in TEST -> {y_test.value_counts()[1]}')
print(f'Ratio of P:N -> {y_test.value_counts()[1]/y_test.value_counts()[0]}')

In [ ]:
# Define the CrossValidation algorithm and parameters 
sss = StratifiedShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 42)

# Missing Data Imputing

In [ ]:
# Counting NaN values in all columns before imputation
nan_count = X_train.isna().sum()

print("\033[1m" + "Missing Values for Each Column \n" + "\033[0m")
print(nan_count)

In [ ]:
# Initialize the KNNImputer with 1 neighbor for imputation of categorical columns
imputer_categorical = KNNImputer(n_neighbors = 1) 

# Impute missing values in the categorical columns for TRAIN data
X_train[categorical_columns] = imputer_categorical.fit_transform(X_train)[ : , categorical_columns_index]

# Save the fitted categorical imputer
joblib.dump(imputer_categorical, cat_imputer_filename)

In [ ]:
# Initialize the KNNImputer with 5 neighbors for imputation of numerical columns
imputer_numerical = KNNImputer(n_neighbors = 5)

# Impute missing values in the numerical columns for TRAIN data
X_imputed_train = imputer_numerical.fit_transform(X_train)
X_imputed_train_df = pd.DataFrame(X_imputed_train, columns = X_train.columns)

# Save the fitted numerical imputer
joblib.dump(imputer_numerical, num_imputer_filename)

In [ ]:
# Set the index of X_train to match the index of y_train
X_imputed_train_df.index = y_train.index

In [ ]:
# Counting NaN values in all columns after imputation for verification
nan_count = X_imputed_train_df.isna().sum()

print("\033[1m" + "Missing Values for Each Column \n" + "\033[0m")
print(nan_count)

# Unbalanced Assessment Copies

In [ ]:
# Create unbalanced copies for final model assessment
X_imputed_train_final = X_imputed_train_df.copy(deep = True)
y_train_final = np.ravel(y_train.copy(deep = True))

# Data Balancing

In [ ]:
# Preview of positive and negative cases in Training set
print(f'Number of NEGATIVE cases in TRAIN -> {y_train.value_counts()[0]}')
print(f'Number of POSITIVE cases in TRAIN -> {y_train.value_counts()[1]}')
print(f'Ratio of P:N -> {y_train.value_counts()[1]/y_train.value_counts()[0]}')

In [ ]:
# Balancing the classes for the training dataframe using SVMSMOTE
X_imputed_train_df, y_train = SVMSMOTE(sampling_strategy = 1, random_state = 42).fit_resample(X_imputed_train_df, y_train)

# Print new numbers for Training set
print(f'Number of NEGATIVE cases in TRAIN -> {y_train.value_counts()[0]}')
print(f'Number of POSITIVE cases in TRAIN -> {y_train.value_counts()[1]}')
print(f'Ratio of P:N -> {y_train.value_counts()[1]/y_train.value_counts()[0]}')

In [ ]:
# Visualization of balanced data frequency
plot_values = y_train.value_counts(ascending = True)
fig, ax = plt.subplots()

ax.bar(x = ['0', '1'], height = plot_values.array)

plt.title('Balanced Distribution of Targets in Train Data using Synthetic Data\n')
plt.xlabel('Toxicity Class')
plt.ylabel('n')
plt.show()

# StandardScaler and TargetEncoder

In [ ]:
# Pre-processing of the numerical features using a StandardScaler
scaler = StandardScaler()

numerical_columns = ['BMI', 'AgeAtCompletion', 'TreatmentDays', 'V100', 'D50', 'D90', 'D98', 'D0.1Bladder', 'D1Bladder',
    'D2Bladder', 'D0.1SmallBowel', 'D1SmallBowel', 'D2SmallBowel',
    'D0.1Sigmoid', 'D1Sigmoid', 'D2Sigmoid','D0.1Rectum', 'D1Rectum', 'D2Rectum',
    'Size', 'HRCTVolume', 'FollowUp']

categorical_columns = ['Chemotherapy', 'Charlson', 'KPS', 'Ethnicity', 'TypeOfBoost', 'Applicator', 'MRI', 
    'TumorSite', 'Stage', 'Histology']

X_imputed_train_df[numerical_columns] = scaler.fit_transform(X_imputed_train_df[numerical_columns])

# Save the fitted StandardScaler using joblib
joblib.dump(scaler, scaler_filename)

In [ ]:
# Initialize the TargetEncoder
target_encoder = TargetEncoder(cols = categorical_columns)

# Fit and transform the categorical variables
X_imputed_train_df[categorical_columns] = target_encoder.fit_transform(X_imputed_train_df[categorical_columns], y_train)

# Save the fitted TargetEncoder
joblib.dump(target_encoder, encoder_filename)

# Drop Colinear Features

In [ ]:
# Find co-linear features using Pearson Correlation Coefficient
corr_matrix = X_imputed_train_df.corr().abs()

# Pick up the upper triangle of our co-linearity matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

print(upper)

In [ ]:
# Find features with correlation greater than 0.85 and store them in a List
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]

# Print the variables that are co-linear
print(to_drop)

In [ ]:
# OPTIONAL: Print the co-lineariaty values for the variable you are planning to drop to see which variables it is co-linear with
print(corr_matrix[['D98']]) 

In [ ]:
# Determine which co-linear variables are appropiate to drop
to_drop = ['D98','D1Bladder','D0.1SmallBowel', 'D1SmallBowel','D1Sigmoid', 'D0.1Rectum', 'D1Rectum']  

In [ ]:
# Drop features for TRAINING set
X_imputed_train_df = X_imputed_train_df.drop(to_drop, axis=1, inplace=False)

# Print Final DB Columns
X_imputed_train_df.columns.tolist()

In [ ]:
# Check the variance for remaining columns. Variance close to 0 means values are almost constant and may be considered to be dropped.
X_imputed_train_df.var(0)

# Load external instances (optional)

In [ ]:
# Load all fitted transformers - if needed
imputer_categorical = joblib.load(cat_imputer_filename)

imputer_numerical = joblib.load(num_imputer_filename)

scaler = joblib.load(scaler_filename)

target_encoder = joblib.load(encoder_filename)

# Preparing datasets for final fit

In [ ]:
# Transform the numerical columns in the assesment training data (unbalanced)
X_imputed_train_final[numerical_columns] = scaler.transform(X_imputed_train_final[numerical_columns])

# Apply the TargetEncoder to the categorical columns in the assesment training data (unbalanced)
X_imputed_train_final[categorical_columns] = target_encoder.transform(X_imputed_train_final[categorical_columns])

# Drop features for the assesment training data (unbalanced)
X_imputed_train_final = X_imputed_train_final.drop(to_drop, axis = 1, inplace = False)

In [ ]:
# Preprocess test data with the previously fitted transformers 
X_test[categorical_columns] = imputer_categorical.transform(X_test)[ : , categorical_columns_index]
X_imputed_test = imputer_numerical.transform(X_test)

X_imputed_test_df = pd.DataFrame(X_imputed_test, columns = X_test.columns)
X_imputed_test_df.index = y_test.index

X_imputed_test_df[numerical_columns] = scaler.transform(X_imputed_test_df[numerical_columns])
X_imputed_test_df[categorical_columns] = target_encoder.transform(X_imputed_test_df[categorical_columns])

# Drop features for the Testing data
X_imputed_test_df = X_imputed_test_df.drop(to_drop, axis = 1, inplace = False)

In [ ]:
# Counting NaN values in final datasets
nan_count_assesment = X_imputed_train_final.isna().sum()
print("\033[1m" + "Missing Values for Each Column in Final Training Set\n" + "\033[0m")
print(nan_count_assesment)

nan_count = X_imputed_test_df.isna().sum()
print("\n\033[1m" + "Missing Values for Each Column in Testing Set\n" + "\033[0m")
print(nan_count)

In [ ]:
# Changing the format of our Y label so that it is compatible with the Sequential Feature Selection method.
labels_test = np.ravel(y_test)
labels_train = np.ravel(y_train)

# Hyperparameter Tuning

## Logistic Regression

In [ ]:
# Instances
lr1 = LogisticRegression(random_state = 42, max_iter = 250)
lr2 = LogisticRegression(random_state = 42, max_iter = 250)

# Sequential Feature Selection
sfsLR = SFS(estimator = lr1, 
    k_features = 'best',
    forward = True, 
    floating = False, 
    scoring = 'f1',
    cv = sss)

# Pipeline
pipe_lr = Pipeline([
    ('lr2', lr2)])

# Hyperparameters to explore
param_range_fl = [1.0, 0.7, 0.5, 0.3, 0.1]

lr_param_grid = [{'lr2__penalty': ['l1', 'l2'],
    'lr2__C': param_range_fl,
    'lr2__solver': ['liblinear','lbfgs']}]

# Gridsearch will use the parameters defined on param_grids to find the best Hyper-Parameter for the model.
lr_grid_search = GridSearchCV(estimator = pipe_lr,
    param_grid = lr_param_grid,
    scoring = 'f1',
    cv = sss)

## Random Forest

In [ ]:
# Instances
rf2 = RandomForestClassifier(random_state = 42)

# SFS placeholder - needed for "zip" function used to calculate metrics afterwards
sfsRF = 'placeholder'

# Pipeline
pipe_rf = Pipeline([
    ('rf2', rf2)])

# Hyperparameters to explore
param_range_rf = [5, 6, 7, 8, 9, 10]
param_estimators_rf = [10, 15, 20, 25, 30, 35, 40, 45, 50]
param_max_features = ['sqrt', 'log2']

rf_param_grid = [{'rf2__min_samples_leaf': param_range_rf,
        'rf2__n_estimators': param_estimators_rf,
        'rf2__min_samples_split': param_range_rf,
        'rf2__max_features': param_max_features}]

# Gridsearch will use the parameters defined on param_grids to find the best Hyper-Parameter for the model.
rf_grid_search = GridSearchCV(estimator = pipe_rf,     
    param_grid = rf_param_grid,
    scoring = 'f1',
    cv = sss)

## K-Nearest Neighbors

In [ ]:
# Instances
knn1 = KNeighborsClassifier()
knn2 = KNeighborsClassifier()

# Sequential Feature Selection
sfsKNN = SFS(estimator = knn1, 
    k_features = 'best',
    forward = True, 
    floating = False, 
    scoring = 'f1',
    cv = sss)

# Pipeline
pipe_knn = Pipeline([
    ('knn2', knn2)])

# Hyperparameters to explore
param_range_knn = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

knn_param_grid = [{'knn2__n_neighbors': param_range_knn}]

# Gridsearch will use the parameters defined on param_grids to find the best Hyper-Parameter for the model.
knn_grid_search = GridSearchCV(estimator = pipe_knn,
    param_grid = knn_param_grid,
    scoring = 'f1',
    cv = sss)

## Support Vector Machines

In [ ]:
# Instances
svm1 = SVC(random_state = 42, probability = True)
svm2 = SVC(random_state = 42, probability = True)

# Sequential Feature Selection
sfsSVM = SFS(estimator = svm1, 
    k_features = 'best',
    forward = True, 
    floating = False, 
    scoring = 'f1',
    cv = sss)

# Pipeline
pipe_svm = Pipeline([
    ('svm2',svm2)])

# Hyperparameters to explore
param_range_svm = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

svm_param_grid = [{'svm2__kernel': ['linear', 'rbf','poly','sigmoid'],
    'svm2__gamma': ['scale', 'auto'], 
    'svm2__C': param_range_svm}]

# Gridsearch will use the parameters defined on param_grids to find the best Hyper-Parameter for the model.
svm_grid_search = GridSearchCV(estimator = pipe_svm,
    param_grid = svm_param_grid,
    scoring = 'f1',
    cv = sss)

## Gaussian Naive Bayes

In [ ]:
# Instances
gnb1 = GaussianNB()
gnb2 = GaussianNB()

# Sequential Feature Selection
sfsGNB = SFS(estimator = gnb1, 
    k_features = 'best',
    forward = True, 
    floating = False, 
    scoring = 'f1',
    cv = sss)

# Pipeline
pipe_gnb = Pipeline([
    ('gnb2',gnb2)])

# Hyperparameters to explore
var_smoothingNumbers=[5e-08, 1e-09, 5e-09]

gnb_param_grid = [{'gnb2__var_smoothing': var_smoothingNumbers}]

# Gridsearch will use the parameters defined on param_grids to find the best Hyper-Parameter for the model.
gnb_grid_search = GridSearchCV(estimator = pipe_gnb,
    param_grid = gnb_param_grid,
    scoring = 'f1',
    cv = sss)

## MultiLayer Perceptron

In [ ]:
# Instances
mlp2 = MLPClassifier(hidden_layer_sizes = (16,8), random_state = 42) 

# SFS placeholder - needed for "zip" function used to calculate metrics afterwards
sfsMLP = 'placeholder'

# Pipeline
pipe_mlp = Pipeline([
    ('mlp2', mlp2)])

# Hyperparameters to explore
learning_rates = [0.001, .1, .15, .2, .3, .5, 1]
MLPalpha=[0.0001, 0.0005, 0.001]
MLPbatchsize=[20, 32, 50]
MLPtols=[0.00001, 0.0001, 0.0005]

mlp_param_grid = [{ 'mlp2__learning_rate': ['constant', 'invscaling','adaptive'],
    'mlp2__activation': ['logistic', 'tanh', 'relu'],
    'mlp2__solver': ['lbfgs', 'sgd'],
    'mlp2__learning_rate_init': learning_rates,
    'mlp2__alpha': MLPalpha,
    'mlp2__batch_size': MLPbatchsize,
    'mlp2__tol': MLPtols}]

# Gridsearch will use the parameters defined on param_grids to find the best Hyper-Parameter for the model.
mlp_grid_search = GridSearchCV(estimator = pipe_mlp,     
    param_grid = mlp_param_grid,
    scoring = 'f1',
    cv = sss)

# GridSearch Exploration

In [ ]:
# Models we are exploring
grid_dict = {0: 'Logistic Regression', 1: 'Random Forest', 2: 'K-Nearest Neighbors',
3: 'Support Vector Machines', 4: 'Gaussian Naive Bayes', 5: 'Multi-Layer Perceptron'}

grids = [lr_grid_search, rf_grid_search, knn_grid_search, svm_grid_search, gnb_grid_search, mlp_grid_search]

SFSList = [sfsLR, sfsRF, sfsKNN, sfsSVM, sfsGNB, sfsMLP]

total_time_start = time.time()

j = 0
for pipe, sfs in zip(grids, SFSList):
    
    if sfs == sfsLR or sfs == sfsKNN or sfs == sfsSVM or sfs == sfsGNB:
        
        model_time_start = time.time()
        
        # Fit the SFS to the balanced Train data [This will also update the SFSList items]
        sfs = sfs.fit(X_imputed_train_df, labels_train)

        # Get selected feature indices
        selected_feature_indices = list(sfs.k_feature_idx_)

        # Create a pandas DataFrame with selected features
        sfsFinal = X_imputed_train_df.iloc[:, selected_feature_indices]

        # Rename columns to original feature names
        sfsFinal.columns = X_imputed_train_df.columns[selected_feature_indices]

        # Perform the GridSearch
        pipe.fit(sfsFinal, labels_train)

        print(f'Finished {grid_dict[j]} -> {(time.time() - model_time_start)/60} m' )
        j += 1
        
    else:

        model_time_start = time.time()
        
        # Perform the GridSearch
        pipe.fit(X_imputed_train_df, labels_train)

        print(f'Finished {grid_dict[j]} -> {(time.time() - model_time_start)/60} m' )
        j += 1

print(f'Total time -> {(time.time() - total_time_start)/60} m' )

# Final Model Training

In [ ]:
# Using the best hyperparameters from GridSearch, the final model is fitted for each algorithm
for i, (sfsItem, model) in enumerate(zip(SFSList, grids)):
    
    if sfsItem == sfsLR or sfsItem == sfsKNN or sfsItem == sfsSVM or sfsItem == sfsGNB:
       
        print(f'----- {grid_dict[i]} -----')
        
        # Get the indices of selected features 
        selected_feature_indices = list(sfsItem.k_feature_idx_)
        
        print(f'Best Hyper Parameters -> {model.best_params_}')
        print(f'SFS F1 Score -> {sfsItem.k_score_}')
        print(f'SFS Feature Names -> {sfsItem.k_feature_names_}')
        print(f'Score for Trained Tuned model -> {model.best_score_}\n') 
        
        # Create a pandas DataFrame with selected features
        X_train_selected = X_imputed_train_final.iloc[:, selected_feature_indices]
        X_test_selected = X_imputed_test_df.iloc[:, selected_feature_indices]
       
        # Calculate various metrics
        y_pred_train = model.predict(X_train_selected)
        y_pred_test = model.predict(X_test_selected)
    
        f1_train = f1_score(y_train_final, y_pred_train)
        accuracy_train = accuracy_score(y_train_final, y_pred_train)
        precision_train = precision_score(y_train_final, y_pred_train)
        recall_train = recall_score(y_train_final, y_pred_train)

        print(f'Train Data F1 Score: -> {f1_train}')
        print(f'Train Data Accuracy: -> {accuracy_train}')
        print(f'Train Data Precision: -> {precision_train}')
        print(f'Train Data Recall: -> {recall_train}\n')
        
        f1_test = f1_score(labels_test, y_pred_test)
        accuracy_test = accuracy_score(labels_test, y_pred_test)
        precision_test = precision_score(labels_test, y_pred_test)
        recall_test = recall_score(labels_test, y_pred_test)

        print(f'Test Data F1 Score: -> {f1_test}')
        print(f'Test Data Accuracy: -> {accuracy_test}')
        print(f'Test Data Precision: -> {precision_test}')
        print(f'Test Data Recall: -> {recall_test}\n')

        # Calculate AUC-ROC and AUC-PR scores
        fpr_train, tpr_train, _ = roc_curve(y_train_final, model.predict_proba(X_train_selected)[:, 1])
        auc_roc_train = auc(fpr_train, tpr_train)

        precision_train, recall_train, _ = precision_recall_curve(y_train_final, model.predict_proba(X_train_selected)[:, 1])
        auc_pr_train = auc(recall_train, precision_train)

        print(f'Train Data AUC-ROC Score -> {auc_roc_train}')
        print(f'Train Data AUC-PR Score- > {auc_pr_train}')
        
        fpr_test, tpr_test, _ = roc_curve(labels_test, model.predict_proba(X_test_selected)[:, 1])
        auc_roc_test = auc(fpr_test, tpr_test)

        precision_test, recall_test, _ = precision_recall_curve(labels_test, model.predict_proba(X_test_selected)[:, 1])
        auc_pr_test = auc(recall_test, precision_test)

        print(f'Test Data AUC-ROC Score -> {auc_roc_test}')
        print(f'Test Data AUC-PR Score- > {auc_pr_test}')
        print('\n')
        
    else:
        
        print(f'----- {grid_dict[i]} -----')
        
        print(f'Score for Trained Tuned model -> {model.best_score_}\n') 
        
        # Rename final sets for code consistency
        X_train_selected = X_imputed_train_final
        X_test_selected = X_imputed_test_df
        
        # Calculate various metrics
        y_pred_train = model.predict(X_train_selected)
        y_pred_test = model.predict(X_test_selected)
    
        f1_train = f1_score(y_train_final, y_pred_train)
        accuracy_train = accuracy_score(y_train_final, y_pred_train)
        precision_train = precision_score(y_train_final, y_pred_train)
        recall_train = recall_score(y_train_final, y_pred_train)

        print(f'Train Data F1 Score: -> {f1_train}')
        print(f'Train Data Accuracy: -> {accuracy_train}')
        print(f'Train Data Precision: -> {precision_train}')
        print(f'Train Data Recall: -> {recall_train}\n')
        
        f1_test = f1_score(labels_test, y_pred_test)
        accuracy_test = accuracy_score(labels_test, y_pred_test)
        precision_test = precision_score(labels_test, y_pred_test)
        recall_test = recall_score(labels_test, y_pred_test)

        print(f'Test Data F1 Score: -> {f1_test}')
        print(f'Test Data Accuracy: -> {accuracy_test}')
        print(f'Test Data Precision: -> {precision_test}')
        print(f'Test Data Recall: -> {recall_test}\n')

        # Calculate AUC-ROC and AUC-PR scores
        fpr_train, tpr_train, _ = roc_curve(y_train_final, model.predict_proba(X_train_selected)[:, 1])
        auc_roc_train = auc(fpr_train, tpr_train)

        precision_train, recall_train, _ = precision_recall_curve(y_train_final, model.predict_proba(X_train_selected)[:, 1])
        auc_pr_train = auc(recall_train, precision_train)

        print(f'Train Data AUC-ROC Score -> {auc_roc_train}')
        print(f'Train Data AUC-PR Score- > {auc_pr_train}')
        
        fpr_test, tpr_test, _ = roc_curve(labels_test, model.predict_proba(X_test_selected)[:, 1])
        auc_roc_test = auc(fpr_test, tpr_test)

        precision_test, recall_test, _ = precision_recall_curve(labels_test, model.predict_proba(X_test_selected)[:, 1])
        auc_pr_test = auc(recall_test, precision_test)

        print(f'Test Data AUC-ROC Score -> {auc_roc_test}')
        print(f'Test Data AUC-PR Score- > {auc_pr_test}')
        print('\n')

# Plot Precision-Recall Curve

In [ ]:
# Create a list to store the selected features for each model
X_test_selected_list = []

for i, (sfsItem, model) in enumerate(zip(SFSList, grids)):
    
    if sfsItem == sfsLR or sfsItem == sfsKNN or sfsItem == sfsSVM or sfsItem == sfsGNB:
        # Get the indices of selected features from the current SFS object
        selected_feature_indices = list(sfsItem.k_feature_idx_)

        # Create a pandas DataFrame with selected features
        X_test_selected = X_imputed_test_df.iloc[:, selected_feature_indices]

        # Append X_test_selected to the list
        X_test_selected_list.append(X_test_selected)
    else:
        # Rename final set for code consistency
        X_test_selected = X_imputed_test_df

        # Append X_test_selected to the list
        X_test_selected_list.append(X_test_selected)

# Calculate the "No Skill" line based on your data
no_skill = len(labels_test[labels_test == 1]) / len(labels_test)
line_styles = ['-', '--', '-.', ':', '-', '--', '-.', ':']

# Plot the Precision-Recall curves for all models
plt.figure(figsize=(10, 6))
for i, (sfsItem, model) in enumerate(zip(SFSList, grids)):
    precision, recall, _ = precision_recall_curve(labels_test, model.predict_proba(X_test_selected_list[i])[:, 1])
    
    # Calculate AUC
    auc_pr = auc(recall, precision)
    
    # Cycle through line styles using modulo operator
    linestyle = line_styles[i % len(line_styles)]
    
    plt.plot(recall, precision, label='Model {} (AUC = {:.2f})'.format(grid_dict[i], auc_pr), linestyle=linestyle)

# Plot the "No Skill" line
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', color='black', label='No Skill')

plt.xlabel('Recall', fontsize='x-large')
plt.ylabel('Precision', fontsize='x-large',labelpad=10)
plt.title('Precision-Recall Curve for All Models', fontsize=15)
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

# Plot formatting
plt.legend(loc='upper center', bbox_to_anchor=(0.496, -0.15), fancybox=True, shadow=True, ncol=2, fontsize='large')

plt.show()

# Plot ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc

# Create a list to store the selected features for each model
X_test_selected_list = []

for i, (sfsItem, model) in enumerate(zip(SFSList, grids)):
    
    if sfsItem == sfsLR or sfsItem == sfsKNN or sfsItem == sfsSVM or sfsItem == sfsGNB:
        # Get the indices of selected features from the current SFS object
        selected_feature_indices = list(sfsItem.k_feature_idx_)

        # Create a pandas DataFrame with selected features
        X_test_selected = X_imputed_test_df.iloc[:, selected_feature_indices]

        # Append X_test_selected to the list
        X_test_selected_list.append(X_test_selected)
    else:
        # Rename final set for code consistency
        X_test_selected = X_imputed_test_df

        # Append X_test_selected to the list
        X_test_selected_list.append(X_test_selected)

# Calculate the "No Skill" line based on your data
no_skill = len(labels_test[labels_test == 1]) / len(labels_test)
line_styles = ['-', '--', '-.', ':', '-', '--', '-.', ':']

# Plot the ROC curves for all models
plt.figure(figsize=(10, 6))
for i, (sfsItem, model) in enumerate(zip(SFSList, grids)):
    fpr, tpr, _ = roc_curve(labels_test, model.predict_proba(X_test_selected_list[i])[:, 1])
    
    # Calculate AUC
    roc_auc = auc(fpr, tpr)
    
    # Cycle through line styles using modulo operator
    linestyle = line_styles[i % len(line_styles)]
    
    plt.plot(fpr, tpr, label='Model {} (AUC = {:.2f})'.format(grid_dict[i], roc_auc), linestyle=linestyle)

# Plot the "No Skill" line
plt.plot([0, 1], [0, 1], linestyle='--', color='black', label='No Skill')

plt.xlabel('False Positive Rate', fontsize='x-large')
plt.ylabel('True Positive Rate', fontsize='x-large',labelpad=10)
plt.title('ROC Curve for All Models', fontsize=15)
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

# Plot formatting
plt.legend(loc='upper center', bbox_to_anchor=(0.496, -0.15), fancybox=True, shadow=True, ncol=2, fontsize='large')

plt.show()